# Tâche 4

In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.ext.matproj import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pymatgen.io.cif import CifParser


In [2]:
mp_key = "5KTRCxoFyWvECvvRVCS0OeflSzkuO7df"
mp_id = "mp-7575"

In [4]:

with MPRester(mp_key) as m:
    structure = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(structure).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

## Vecteurs des réseaux direct et réciproque

In [14]:
# Vecteurs du réseau direct (cellule primitive)
lattice = structure.lattice
direct_vectors = lattice.matrix
a, b, c = direct_vectors[0], direct_vectors[1], direct_vectors[2]
print("Vecteurs du réseau direct (cellule primitive) (Å):")
print(f"a = {a}, b = {b}, c = {c} \n")
norme = np.linalg.norm(direct_vectors, axis = 1)
print("La norme de ces 3 vecteurs est ", norme[0], "Å", " ce qui correspond bien aux paramètres de maille de la cellule primitive (a,b,c) d'après le Materials Project.")


# Vecteurs du réseau réciproque (cellule primitive)
reciprocal_lattice = lattice.reciprocal_lattice
a_r, b_r, c_r = reciprocal_lattice.matrix[0], reciprocal_lattice.matrix[1], reciprocal_lattice.matrix[2]
print("\nVecteurs du réseau réciproque (cellule primitive) (Å^-1):")
print(f"a* = {a_r}, b* = {b_r}, c* = {c_r}")


# Structure conventionnelle
analyzer = SpacegroupAnalyzer(structure)
conventional_structure = analyzer.get_conventional_standard_structure()

# Cellule conventionnelle
conventional_lattice = conventional_structure.lattice
a, b, c = conventional_lattice.matrix[0], conventional_lattice.matrix[1], conventional_lattice.matrix[2]
print("\nVecteurs du réseau direct (cellule conventionnelle) (Å):")
print(f"a = {a}, b = {b}, c = {c}")
norme = np.linalg.norm(conventional_lattice.matrix, axis = 1)
print("La norme de ces 3 vecteurs est ", norme[0], "Å", " ce qui correspond bien aux paramètres de maille de la cellule conventionnelle (a,b,c) d'après le Materials Project.")


# Réseau réciproque liés aux vecteurs de base de la cellule conventionnelle
reciprocal_lattice = conventional_lattice.reciprocal_lattice
a_r, b_r, c_r = reciprocal_lattice.matrix[0], reciprocal_lattice.matrix[1], reciprocal_lattice.matrix[2]
print("\nVecteurs du réseau réciproque (cellule conventionnelle) (Å^-1):")
print(f"a* = {a_r}, b* = {b_r}, c* = {c_r}")



Vecteurs du réseau direct (cellule primitive) (Å):
a = [2.97906657 0.         1.71996563], b = [0.99302253 2.80869047 1.71996563], c = [0.         0.         3.43992825] 

La norme de ces 3 vecteurs est  3.439930144185033 Å  ce qui correspond bien aux paramètres de maille de la cellule primitive (a,b,c) d'après le Materials Project.

Vecteurs du réseau réciproque (cellule primitive) (Å^-1):
a* = [ 2.10911209 -0.7456841   0.        ], b* = [0.         2.23705153 0.        ], c* = [-1.05455697 -0.74568437  1.82654545]

Vecteurs du réseau direct (cellule conventionnelle) (Å):
a = [4.86479437e+00 0.00000000e+00 2.97882743e-16], b = [7.82318602e-16 4.86479437e+00 2.97882743e-16], c = [0.         0.         4.86479437]
La norme de ces 3 vecteurs est  4.864794373012699 Å  ce qui correspond bien aux paramètres de maille de la cellule conventionnelle (a,b,c) d'après le Materials Project.

Vecteurs du réseau réciproque (cellule conventionnelle) (Å^-1):
a* = [ 1.29156236e+00 -2.07699069e-16  0.00

## Type de maille, système cristallin et groupe ponctuel

In [19]:
#Type de maille
maille_type = analyzer.get_space_group_symbol()[0]

print("\nType de maille:", maille_type)
print("La maille est donc faces centrées.")

#Système cristallin
crystal_system = analyzer.get_crystal_system()
print("\nSystème cristallin:", crystal_system)
print("Le type de réseau de Bravais est donc cubique faces centrées.")

#Groupe ponctuel:
point_group_symbol = analyzer.get_point_group_symbol()
print("\nGroupe ponctuel:", point_group_symbol)
print("Il s'agit du groupe reprenant l'ensemble des opérations ponctuelles de symétrie du cristal.")
print("-4 représente une rotoinversion d'ordre 4 le long des directions [100],[010] et [001].")
print("3 représente une rotation d'ordre 3 le long des directions [111], [1-1-1], [-11-1] et [-1-11].")
print("m représente la réflexion à travers un plan miroir le long des directions [110], [101], [011], [1-10], [10-1] et [01-1].")


Type de maille: F
La maille est donc faces centrées.

Système cristallin: cubic
Le type de réseau de Bravais est donc cubique faces centrées.

Groupe ponctuel: -43m
Il s'agit du groupe reprenant l'ensemble des opérations ponctuelles de symétrie du cristal.
-4 représente une rotoinversion d'ordre 4 le long des directions [100],[010] et [001].
3 représente une rotation d'ordre 3 le long des directions [111], [1-1-1], [-11-1] et [-1-11].
m représente la réflexion à travers un plan miroir le long des directions [110], [101], [011], [1-10], [10-1] et [01-1].
